In [3]:
import json
from eval.eval_molopt import eval_molopt_from_list
print("success")
    

success


In [5]:
def tranform_str_to_json(str_input):
    ## 假如LLM输出的是类似json的字符串, 我需要设定一个逻辑, 把字符串重新转换成json
    ## o1-mini的感觉, 是要移除字符串里面的\n，并且把所有的\"都改成 "
    if "</think>\n\n" in str_input:
        str_input = str_input.split("</think>\n\n")[-1]
        
    if "```json\n" in str_input:
        str_input = str_input.split("```json\n")[1]
        str_input = str_input.replace("\n```", '')
    
    unescaped_str = str_input.replace('\n    ', '').replace('\n', '').replace('\"', '"')
    try:
        json_obj = json.loads(unescaped_str)
        return json_obj
    except json.JSONDecodeError as e:
        return None

def evaluate_molopt_score(model_name=None):
    ## 在get_molopt_cot中得到test结果, 我们评测这些test结果
    prop_dict = dict(logp='logp', solubility='solubility', qed="qed",  drd='drd2', jnk='jnk3', gsk='gsk3b')
    # prop_dict = dict(logp='logp', solubility='solubility', qed="qed",  drd='drd2', gsk='gsk3b')
    
    result_final = dict()
    
    for prop in prop_dict.keys():
        print(model_name, prop)
        file_name = f"logs/{prop}/{model_name}.json"
        pred_results = json.load(open(file_name, "r"))
        
        tgt_smiles_list, src_smiles_list = list(), list()
        
        invalid_number = 0
        final_target_key = 'Final Target Molecule'
        
        if model_name not in ['biomedgpt', 'biomistral']:
            src_smiles_key =  'src_smiles'
        else: src_smiles_key = "src"
        
        for pred in pred_results:
            ## 提取 predicted-smiles, 如果生成的是json格式那不需要额外操作, 如果不是, 需要转换成json形式
            if type(pred['json_results']) is str:
                pred_json = tranform_str_to_json(str_input=pred['json_results'])
                # if model_name == 'gemini': pred_json = pred_json[0]
                if pred_json == None or type(pred_json) is str:
                    invalid_number += 1
                    continue
                else:
                    if final_target_key in pred_json.keys():
                        tgt_smiles_list.append(pred_json[final_target_key])
                        src_smiles_list.append(pred[src_smiles_key])
            else:
                if final_target_key in pred['json_results'].keys():
                    tgt_smiles_list.append(pred['json_results'][final_target_key])
                    src_smiles_list.append(pred[src_smiles_key])
        
        print(len(pred_results), invalid_number, len(src_smiles_list))
        assert len(src_smiles_list) == len(tgt_smiles_list)
        assert len(pred_results) == invalid_number + len(src_smiles_list)
        
        result_dict = eval_molopt_from_list(optimized_prop=prop, gt_list=src_smiles_list, pred_list=tgt_smiles_list, total_number=len(pred_results))
        result_final[prop] = result_dict
    
    print(f"eval_score_{model_name}", result_final)
    # json.dump(result_final, open(f"../api_results/molopt_caohe/eval_score_{model_name}.json", "w"), indent=4)
        
if __name__ == "__main__":
    for model_name in ['claude3.7']:   # your model name
        evaluate_molopt_score(model_name=model_name)

claude3.7 logp
100 0 100


100%|██████████| 9.05M/9.05M [00:03<00:00, 2.41MiB/s]
Done!


claude3.7 solubility
100 0 100
claude3.7 qed
100 0 100


claude3.7 drd
100 4 96


100%|██████████| 35.4M/35.4M [00:15<00:00, 2.30MiB/s]
Done!


claude3.7 jnk
100 0 100


100%|██████████| 10.9M/10.9M [00:05<00:00, 1.94MiB/s]
Done!


claude3.7 gsk
100 2 98


100%|██████████| 27.8M/27.8M [00:11<00:00, 2.34MiB/s]
Done!


eval_score_claude3.7 {'logp': {'improvement': {'mean': 0.4069006780846787, 'variance': 0.21206145544050284, 'min': -0.21740861942726167, 'max': 1.4166426591449985, 'success_rate': 0.8, 'best_rate': 0.32, 'valid_smiles_rate': 0.99, 'valid_score_rate': 0.99, 'valid_smiles_extract_rate': 1.0}, 'scaffold': {'hard': 0.72, 'soft': 0.9038459342980477}}, 'solubility': {'improvement': {'mean': 0.3705101410563746, 'variance': 0.2923881924941273, 'min': -0.6028324228643849, 'max': 1.4974010988486226, 'success_rate': 0.75, 'best_rate': 0.32, 'valid_smiles_rate': 0.92, 'valid_score_rate': 0.92, 'valid_smiles_extract_rate': 1.0}, 'scaffold': {'hard': 0.79, 'soft': 0.8783154572632265}}, 'qed': {'improvement': {'mean': 0.11797018528900438, 'variance': 0.017880429209160168, 'min': -0.07652908388710614, 'max': 0.346179682568043, 'success_rate': 0.73, 'best_rate': 0.14, 'valid_smiles_rate': 0.94, 'valid_score_rate': 0.94, 'valid_smiles_extract_rate': 1.0}, 'scaffold': {'hard': 0.66, 'soft': 0.81894914650